# IMPORT_ANTS

In [3]:
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression

from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

# Cleaning Data

In [2]:
def Cleaner(DataFrame):
    df = DataFrame
    
    #1 Filling 'carpark' column empty values with 0
    df['carpark'].fillna(0, inplace=True)
    
    #2 Dropping 'psf', bedroom' and 'bathroom' empty values
    df.dropna(inplace=True)
    
    #3 Dropping duplicate rows
    df.drop_duplicates(inplace=True)
    
    #4 Reset index
    df.reset_index(drop=True, inplace=True)
    
    #===== CHECKPOINT : NO MORE NULL & DUPLICATE VALUES =====#
    
    #5 Splitting 'description' column into 'type' and 'area' columns
    if "type" not in df.columns: 
        df.insert(5, "type", None)
    if "area" not in df.columns: 
        df.insert(6, "area", None)
    for x in df.index:
        descriptions = df.loc[x,'description'].split('\xa0•\xa0')
        if len(descriptions) >= 1:
            df.loc[x,'type'] = descriptions[0]
        if len(descriptions) >= 2:
            df.loc[x,'area'] = descriptions[1]
    df.drop(['description'], axis=1, inplace=True)
    
    #6 Splitting 'type' column into 'house_type' and 'extra_detail'
    if "house_type" not in df.columns: 
        df.insert(5, "house_type", None)
    if "extra_details" not in df.columns: 
        df.insert(6, "extra_details", None)
    for x in df.index:
        desc = df.loc[x,'type'].split(' | ')
        if len(desc) >= 1:
            df.loc[x,'house_type'] = desc[0].strip()
        if len(desc) >= 2:
            df.loc[x,'extra_details'] = desc[1].strip()
    df.drop(['type'], axis=1, inplace=True)
    
    #7 Splitting 'address' column into 'district' and 'state'
    if "district" not in df.columns: 
        df.insert(4, "district", None)
    if "state" not in df.columns: 
        df.insert(5, "state", None)
    for x in df.index:
        addr = df.loc[x,'address'].split(',')
        df.loc[x,'district'] = addr[0].strip()
        df.loc[x,'state'] = addr[1].strip()
    df.drop(['address'], axis=1, inplace=True)
    
    #===== CHECKPOINT : NO MORE ADDITIONAL COLUMNS =====#
    
    #8 Dealing with 'Studio' type of houses
    for x in df.index:
        if df.loc[x,'bedroom'] == 'Studio':
            df.loc[x,'extra_details'] = 'Studio'
            df.loc[x,'bedroom'] = '1'
            
    #9 Cleaning 'extra_details' column
    for x in df.index:
        if df.loc[x,'extra_details'] == None:
            df.loc[x,'extra_details'] = 'Default'
    
    #===== CHECKPOINT : NO MORE INCORRECT VALUES =====#
    
    #10 Dealing with / Converting numerical columns
    for x in df.index:
        #10.1 price
        df.loc[x,'price'] = re.sub(r'\,','',df.loc[x,'price'])
        df.loc[x,'price'] = re.findall('\d+',df.loc[x,'price'])[0]

        #10.2 area
        df.loc[x,'area'] = re.sub(r'\,','',df.loc[x,'area'])
        df.loc[x,'area'] = re.findall('\d+',df.loc[x,'area'])[0]

        #10.3 psf
        df.loc[x,'psf'] = re.sub(r'\,','',df.loc[x,'psf'])
        df.loc[x,'psf'] = re.findall('\d+.?\d*',df.loc[x,'psf'])[0]
        
    #10.4 bedroom
    if "extra" not in df.columns: 
        df.insert(9, "extra", None)
    for x in df.index:
        rooms = df.loc[x,'bedroom'].split('+')
        if len(rooms) >= 1:
            df.loc[x,'bedroom'] = rooms[0]
        if len(rooms) >= 2:
            df.loc[x,'extra'] = rooms[1]
        else:
            df.loc[x,'extra'] = '0'

    #11 changing to correct data type
    df = df.astype({'price':'int64','area':'int64','psf':'float64','bedroom':'int64','extra':'int64','bathroom':'int64','carpark':'int64'})
    
    #12 rename column
    df.rename(columns={'house_type':'type'}, inplace=True)
    
    #13 save a copy
    df.to_csv('training.csv', index=False)
    
    return df

In [3]:
df_in = pd.read_csv('./data/iProperty_V2.csv')
df_out = Cleaner(df_in)
df_out

,name,price,psf,district,state,type,extra_details,area,bedroom,extra,bathroom,carpark
0,"The Park Sky Residence, Bukit Jalil",1150000,1042.61,Bukit Jalil,Kuala Lumpur,Serviced Residence,Corner lot,1103,3,0,2,2
1,"Bukit Damansara, Damansara Heights",1650000,717.39,Damansara Heights,Kuala Lumpur,2-sty Terrace/Link House,Intermediate,2300,4,0,3,2
2,"The Binjai on the Park, KL City Centre, KLCC",6888888,2140.74,KLCC,Kuala Lumpur,Condominium,Corner lot,3218,3,1,4,3
3,"AMELIA, DESA PARKCITY, Desa ParkCity",2700000,1026.62,Desa ParkCity,Kuala Lumpur,2-sty Terrace/Link House,Default,2630,3,1,4,2
4,"AMELIA, DESA PARKCITY, Desa ParkCity",2800000,1076.92,Desa ParkCity,Kuala Lumpur,2-sty Terrace/Link House,Default,2600,3,1,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...
13217,"Kg Tiong Dalam, Pasir Tumboh, Kota Bharu",397000,275.69,Kota Bharu,Kelantan,Bungalow,Default,1440,4,0,3,0
13218,"Chekok, Kota Bharu",600000,298.80,Kota Bharu,Kelantan,Bungalow,Default,2008,4,0,3,0
13219,"Kampung Padang Bongor, Tanah Merah",175000,159.53,Tanah Merah,Kelantan,1-sty Terrace/Link House,Default,1097,3,0,2,1
13220,"Kampung Kadok Dalam, Kota Bharu",420000,221.05,Kota Bharu,Kelantan,Bungalow,Default,1900,3,0,3,3


In [4]:
df_out.describe()

,price,psf,area,bedroom,extra,bathroom,carpark
count,1.322200e+04,1.322200e+04,1.322200e+04,13222.000000,13222.000000,13222.00000,13222.000000
mean,1.419758e+06,1.154549e+04,6.939761e+03,3.777038,0.239449,3.24013,1.660112
std,1.666868e+07,1.215985e+05,3.524714e+05,1.258830,0.533848,1.52801,1.583482
min,1.234500e+04,3.000000e-02,1.000000e+00,1.000000,0.000000,1.00000,0.000000
25%,4.200000e+05,2.865100e+02,1.126000e+03,3.000000,0.000000,2.00000,0.000000
50%,6.617500e+05,4.083300e+02,1.740000e+03,4.000000,0.000000,3.00000,2.000000
75%,1.200000e+06,5.876050e+02,2.760000e+03,4.000000,0.000000,4.00000,2.000000
max,1.288000e+09,3.000000e+06,3.702785e+07,18.000000,17.000000,20.00000,20.000000


# Training Regression Model

In [29]:
def Trainer(DataFrame):
    df = DataFrame
    df_all = df
    one_hot = pd.get_dummies(df_all[['state','type','extra_details']])
    df_all = df_all.join(one_hot)    
    df_all.drop(['name','district','state','type','extra_details'],axis=1,inplace=True)
    for col in ['price','area']:
        q_low = df_all[col].quantile(0.01)
        q_hi  = df_all[col].quantile(0.99)
        df_all = df_all[(df_all[col] < q_hi) & (df_all[col] > q_low)]
    
    X = df_all.values[:,1:]
    y = df_all.values[:,0]
    
    MMS = MinMaxScaler()
    X_MMS = MMS.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_MMS, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("MSE value : ", mean_squared_error(y_test, y_pred))
    print("RMSE value : ", mean_squared_error(y_test, y_pred, squared=False))
    print("MAE value : ", mean_absolute_error(y_test, y_pred))
    print("MAPE value : ", mean_absolute_percentage_error(y_test, y_pred))
    print("Score : ", model.score(X_test,y_test))
    print("Number of data :", df_all.shape[0])
    print("")
    print("===" * 20)

    
    
    for state in df.state.unique():
        df_state = df[df['state'] == state]
        one_hot = pd.get_dummies(df[['type','extra_details']])
        df_state = df_state.join(one_hot)
        df_state.drop(['name','district','state','type','extra_details'],axis=1,inplace=True)
        for col in ['price','area']:
            q_low = df_state[col].quantile(0.01)
            q_hi  = df_state[col].quantile(0.99)
            df_state = df_state[(df_state[col] < q_hi) & (df_state[col] > q_low)]
        X = df_state.values[:,1:]
        y = df_state.values[:,0]
        
        try:
            MMS = MinMaxScaler()
            X_MMS = MMS.fit_transform(X)
            X_train, X_test, y_train, y_test = train_test_split(X_MMS, y, test_size=0.2, random_state=42)
            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            print("MSE value for", state, " : ", mean_squared_error(y_test, y_pred))
            print("RMSE value for", state, " : ", mean_squared_error(y_test, y_pred, squared=False))
            print("MAE value for", state, " : ", mean_absolute_error(y_test, y_pred))
            print("MAPE value for", state, " : ", mean_absolute_percentage_error(y_test, y_pred))
            print("Score for", state, " : ", model.score(X_test,y_test))
            print("Number of data :", df_state.shape[0])
            print("")
        except:
            continue

In [30]:
df_train_in = pd.read_csv('training.csv')
Trainer(df_train_in)

MSE value :  296355762828.305
RMSE value :  544385.6747089374
MAE value :  342170.6540265366
MAPE value :  0.4426462069778922
Score :  0.6907029689493354
Number of data : 12641

MSE value for Kuala Lumpur  :  755059307676.467
RMSE value for Kuala Lumpur  :  868941.4869117868
MAE value for Kuala Lumpur  :  512827.3081063759
MAPE value for Kuala Lumpur  :  0.3404825322486797
Score for Kuala Lumpur  :  0.8917285383318925
Number of data : 1373

MSE value for Selangor  :  65485661267.47965
RMSE value for Selangor  :  255901.66327611014
MAE value for Selangor  :  168952.73230452166
MAPE value for Selangor  :  0.19651016039644706
Score for Selangor  :  0.9445099109981232
Number of data : 1550

MSE value for Penang  :  55655858892.462524
RMSE value for Penang  :  235914.93995180237
MAE value for Penang  :  157479.67483597755
MAPE value for Penang  :  0.19050353430163777
Score for Penang  :  0.9137364023049217
Number of data : 1274

MSE value for Perak  :  32757042411.8535
RMSE value for Perak 

# Training Gradient Boosting Regression Model

In [8]:
def Trainer_Ensemble(DataFrame):
    df = DataFrame
    df_all = df
    one_hot = pd.get_dummies(df_all[['state','type','extra_details']])
    df_all = df_all.join(one_hot)    
    df_all.drop(['name','district','state','type','extra_details'],axis=1,inplace=True)
    for col in ['price','area']:
        q_low = df_all[col].quantile(0.01)
        q_hi  = df_all[col].quantile(0.99)
        df_all = df_all[(df_all[col] < q_hi) & (df_all[col] > q_low)]
    
    X = df_all.values[:,1:]
    y = df_all.values[:,0]
    
    MMS = MinMaxScaler()
    X_MMS = MMS.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_MMS, y, test_size=0.2, random_state=42)
    
    model = GradientBoostingRegressor(n_estimators=200, max_depth=5, min_samples_split=2, learning_rate=0.1, loss='squared_error')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("MSE value : ", mean_squared_error(y_test, y_pred))
    print("RMSE value : ", mean_squared_error(y_test, y_pred, squared=False))
    print("MAE value : ", mean_absolute_error(y_test, y_pred))
    print("MAPE value : ", mean_absolute_percentage_error(y_test, y_pred))
    print("Score : ", model.score(X_test,y_test))
    print("Number of data :", df_all.shape[0])
    print("")
    print("===" * 20)

    
    
    for state in df.state.unique():
        df_state = df[df['state'] == state]
        one_hot = pd.get_dummies(df[['type','extra_details']])
        df_state = df_state.join(one_hot)
        df_state.drop(['name','district','state','type','extra_details'],axis=1,inplace=True)
        for col in ['price','area']:
            q_low = df_state[col].quantile(0.01)
            q_hi  = df_state[col].quantile(0.99)
            df_state = df_state[(df_state[col] < q_hi) & (df_state[col] > q_low)]
        X = df_state.values[:,1:]
        y = df_state.values[:,0]
        
        try:
            MMS = MinMaxScaler()
            X_MMS = MMS.fit_transform(X)
            X_train, X_test, y_train, y_test = train_test_split(X_MMS, y, test_size=0.2, random_state=42)
            model = GradientBoostingRegressor(n_estimators=200, max_depth=5, min_samples_split=2, learning_rate=0.1, loss='squared_error')
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            print("MSE value for", state, " : ", mean_squared_error(y_test, y_pred))
            print("RMSE value for", state, " : ", mean_squared_error(y_test, y_pred, squared=False))
            print("MAE value for", state, " : ", mean_absolute_error(y_test, y_pred))
            print("MAPE value for", state, " : ", mean_absolute_percentage_error(y_test, y_pred))
            print("Score for", state, " : ", model.score(X_test,y_test))
            print("Number of data :", df_state.shape[0])
            print("")
        except:
            continue

In [9]:
df_train_in = pd.read_csv('training.csv')
Trainer_Ensemble(df_train_in)

MSE value :  1219194202.631808
RMSE value :  34916.961532066445
MAE value :  16471.35008428192
MAPE value :  0.01758943042184518
Score :  0.9987275660052993
Number of data : 12641

MSE value for Kuala Lumpur  :  308098235455.51935
RMSE value for Kuala Lumpur  :  555065.9739666262
MAE value for Kuala Lumpur  :  146824.18827498207
MAPE value for Kuala Lumpur  :  0.037465982683571784
Score for Kuala Lumpur  :  0.9558203627834393
Number of data : 1373

MSE value for Selangor  :  2497842386.8672433
RMSE value for Selangor  :  49978.419211368055
MAE value for Selangor  :  29700.637777226057
MAPE value for Selangor  :  0.022210364363330657
Score for Selangor  :  0.9978834222075916
Number of data : 1550

MSE value for Penang  :  7182555315.807757
RMSE value for Penang  :  84749.9576153744
MAE value for Penang  :  35131.297898803285
MAPE value for Penang  :  0.02462707090343897
Score for Penang  :  0.988867424301498
Number of data : 1274

MSE value for Perak  :  10066246050.5856
RMSE value for 